# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import astropy.units as u
import jax.numpy as jnp
from astropy.time import Time
from astropy.utils.data import download_file

from jorbit import Particle
from jorbit.data.constants import JORBIT_EPHEM_URL_BASE

In [47]:
ind = 1000
t0 = Time("2020-01-01")
reverse_times = t0 - jnp.arange(0, 20.001, 10 * u.hour.to(u.year)) * u.year
names = jnp.load(download_file(JORBIT_EPHEM_URL_BASE + "names.npy"))
x0 = jnp.load(download_file(JORBIT_EPHEM_URL_BASE + "x0.npy"))
v0 = jnp.load(download_file(JORBIT_EPHEM_URL_BASE + "v0.npy"))
pos = jnp.load(download_file(JORBIT_EPHEM_URL_BASE + "reverse_pos.npy"))

In [48]:
particle = Particle(x=x0[ind], v=v0[ind], time=t0, gravity="newtonian solar system")

In [1]:
import astropy.units as u
import jax.numpy as jnp
from astropy.time import Time
from tqdm import tqdm

from jorbit.utils.horizons import get_observer_positions

##########
# get the positions of the geocenter from 20 years ago to 20 years in the future
##########

t0 = Time("2020-01-01")

forward_times = t0 + jnp.arange(0, 20.001, 10 * u.hour.to(u.year)) * u.year
reverse_times = t0 - jnp.arange(0, 20.001, 10 * u.hour.to(u.year)) * u.year

chunk_size=5
f_pos = []
for i in tqdm(range(int(len(forward_times) / chunk_size) + 1)):
    start = i * chunk_size
    end = (i + 1) * chunk_size
    if end > len(forward_times):
        end = len(forward_times)
    f_pos.append(get_observer_positions(forward_times[start:end], "500@399"))
    break
f_pos[0]




  0%|          | 0/3507 [00:00<?, ?it/s]


Array([[-0.1701589 ,  0.89598132,  0.38843568],
       [-0.17734068,  0.89481653,  0.38793097],
       [-0.18451282,  0.89360373,  0.38740542],
       [-0.19167493,  0.892343  ,  0.38685907],
       [-0.19882664,  0.89103442,  0.38629195]], dtype=float64)

In [2]:
chunk_size=5
reverse_pos = []
for i in tqdm(range(int(len(reverse_times) / chunk_size) + 1)):
    start = i * chunk_size
    end = (i + 1) * chunk_size
    if end > len(reverse_times):
        end = len(reverse_times)
    reverse_pos.append(get_observer_positions(reverse_times[start:end], "500@399"))
    break
reverse_pos[0]

  0%|          | 0/3507 [00:00<?, ?it/s]


Array([[-0.1701589 ,  0.89598132,  0.38843568],
       [-0.16296789,  0.89709801,  0.38891952],
       [-0.15576803,  0.89816652,  0.38938244],
       [-0.14855971,  0.89918678,  0.38982443],
       [-0.14134335,  0.9001587 ,  0.39024544]], dtype=float64)

In [ ]:
times = reverse_times
observatories = ["0"] * len(times)

times = Time([t.utc.jd for t in times], format="jd", scale="utc")
sort_indices = jnp.argsort(times)
times = times[sort_indices]
observatories = [observatories[i] for i in sort_indices]

inverse_indices = jnp.argsort(sort_indices)
restored_result = times[inverse_indices]

In [5]:
reverse_times - restored_result

<TimeDelta object: scale='tai' format='jd' value=[ 0.00000000e+00 -1.55220503e-10  1.55220281e-10 ...  1.58605573e-10
  2.76839662e-12 -1.51771096e-10]>